In [ ]:
%%capture
%config Completer.use_jedi = False
%config InlineBackend.figure_formats = ['svg']
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

# Install on Google Colab
import subprocess
import sys

from IPython import get_ipython

install_packages = "google.colab" in str(get_ipython())
if install_packages:
    for package in ["tensorwaves[doc]", "graphviz"]:
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", package]
        )

# Step 1: Create amplitude model

TensorWaves requires you to first formulate a mathematical {class}`.Model` that you want to fit to your data set. When this model is an amplitude model, it is most convenient to construct it with {mod}`ampform`.

This notebook briefly illustrates how to create such an amplitude model with {mod}`ampform` and how to write it to a recipe file that can be understood by {mod}`tensorwaves`. For more control, have a look at {doc}`the usage guides of AmpForm <ampform:usage>`.

In this example, we use the helicity formalism, but you can also use `formalism="canonical-helicity"`. As you can see, we analyze the decay $J/\psi \to \pi^0\pi^0\gamma$ here.

```{admonition} Simplified model: $J/\psi \to f_0\gamma$
---
class: dropdown
---
As {doc}`step3` serves to illustrate usage only, we make the amplitude model here a bit simpler by not allowing $\omega$ resonances (which are narrow and therefore hard to fit). For this reason, we can also limit the {class}`~qrules.settings.InteractionType` to {attr}`~qrules.settings.InteractionType.STRONG`.
```

In [ ]:
import qrules

reaction = qrules.generate_transitions(
    initial_state=("J/psi(1S)", [-1, +1]),
    final_state=["gamma", "pi0", "pi0"],
    allowed_intermediate_particles=["f(0)"],
    allowed_interaction_types=["strong", "EM"],
    formalism="helicity",
)

As a small goodie, you can use [`graphviz`](https://pypi.org/project/graphviz) to {doc}`visualize <qrules:usage/visualize>` the generated graphs:

In [ ]:
from graphviz import Source

dot = qrules.io.asdot(reaction, collapse_graphs=True)
Source(dot)

Next we convert the {attr}`~qrules.transition.ReactionInfo.transitions` into an amplitude model (here: {class}`~ampform.helicity.HelicityModel`). This can be done with {func}`~ampform.get_builder` and {meth}`~ampform.helicity.HelicityAmplitudeBuilder.formulate`.

In [ ]:
import ampform

model_builder = ampform.get_builder(reaction)
model = model_builder.formulate()
display(*model.parameter_defaults)

The heart of the model is a sympy expression that contains the full description of the intensity model. Note two things:
1. The coefficients for the different amplitudes are **complex** valued.
2. By default there is no dynamics in the model, so it still has to be specified.

We choose to use {func}`~ampform.dynamics.relativistic_breit_wigner_with_ff` as the lineshape for all resonances and use a Blatt-Weisskopf form factor (no dynamics; see {class}`~ampform.dynamics.BlattWeisskopfSquared`) for the production decay. The {meth}`~ampform.helicity.HelicityAmplitudeBuilder.set_dynamics` is a convenience interface for replacing the dynamics for intermediate states.

In [ ]:
from ampform.dynamics.builder import (
    create_non_dynamic_with_ff,
    create_relativistic_breit_wigner_with_ff,
)

model_builder.set_dynamics("J/psi(1S)", create_non_dynamic_with_ff)
for name in reaction.get_intermediate_particles().names:
    model_builder.set_dynamics(name, create_relativistic_breit_wigner_with_ff)
model = model_builder.formulate()

Now let's take another look at the parameters of the model to see which new parameters are there:

In [ ]:
sorted(model.parameter_defaults, key=lambda s: s.name)

Finally, we can write the {class}`~ampform.helicity.HelicityModel` to disk via {mod}`pickle`, as well as store the {class}`~qrules.transition.ReactionInfo` as JSON:

In [ ]:
import pickle

qrules.io.write(reaction, "transitions.json")
with open("helicity_model.pickle", "wb") as stream:
    pickle.dump(model, stream)

Cool, that's it! We now have a template for an amplitude model with which to {doc}`generate data <step2>` and {doc}`perform a fit <step3>`. In the next steps, we will use use this {class}`~ampform.helicity.HelicityModel` as a fit model template for {mod}`tensorwaves`.